In [2]:
# Adult Income — Baseline training + fairness metrics
# Input :  data/adult_model.csv  (features + binary target 'label')
# Output:  results/metrics/adult_overall_metrics.csv
#          results/metrics/adult_group_metrics.csv
#          models/adult/best_pipeline.joblib

from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import joblib

# Paths
project_root = Path.cwd().resolve().parent  # run from notebooks/
data_dir = project_root / "data"
results_dir = project_root / "results" / "metrics"
models_dir = project_root / "models" / "adult"
results_dir.mkdir(parents=True, exist_ok=True)
models_dir.mkdir(parents=True, exist_ok=True)

# Load
df = pd.read_csv(data_dir / "adult_model.csv")

# Target and features
y = df["label"].astype(int).values
X = df.drop(columns=["label"]).copy()

# Sensitive attributes (if present)
sensitive_cols = [c for c in ["sex", "race"] if c in X.columns]
sensitive_df = X[sensitive_cols].copy() if sensitive_cols else pd.DataFrame(index=X.index)

# Split
X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X, y, sensitive_df, test_size=0.25, random_state=42, stratify=y
)

# Columns by type
num_cols = [c for c in X_train.columns if np.issubdtype(X_train[c].dtype, np.number)]
cat_cols = [c for c in X_train.columns if c not in num_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), cat_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0,
)

models = {
    "logreg": LogisticRegression(max_iter=1000, random_state=42),
    "rf": RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
}

def eval_fairness(model_name, pipe, X_te, y_te, sens_te):
    y_pred = pipe.predict(X_te)
    try:
        y_proba = pipe.predict_proba(X_te)[:, 1]
        roc_auc = roc_auc_score(y_te, y_proba)
    except Exception:
        roc_auc = np.nan

    overall = {
        "model": model_name,
        "accuracy": accuracy_score(y_te, y_pred),
        "f1": f1_score(y_te, y_pred),
        "roc_auc": roc_auc,
    }

    rows = []
    if not sens_te.empty:
        def tpr(y_true, y_hat):
            pos = (y_true == 1).sum()
            return ((y_true == 1) & (y_hat == 1)).sum() / pos if pos > 0 else np.nan

        for attr in sens_te.columns:
            groups = sens_te[attr].astype(str).unique()
            per_group = []
            for g in sorted(groups):
                mask = sens_te[attr].astype(str) == g
                yp_g = y_pred[mask]
                yt_g = y_te[mask]
                p_pos = (yp_g == 1).mean() if len(yp_g) else np.nan
                acc = accuracy_score(yt_g, yp_g) if len(yt_g) else np.nan
                tpr_g = tpr(yt_g, yp_g) if len(yt_g) else np.nan
                per_group.append({"p_pos": p_pos, "tpr": tpr_g})

                rows.append({
                    "model": model_name, "attribute": attr, "group": g,
                    "n": int(mask.sum()), "p_pos": p_pos, "accuracy": acc, "tpr": tpr_g
                })

            dp_gap = np.nanmax([r["p_pos"] for r in per_group]) - np.nanmin([r["p_pos"] for r in per_group])
            eopp_gap = np.nanmax([r["tpr"] for r in per_group]) - np.nanmin([r["tpr"] for r in per_group])
            overall[f"{attr}_dp_gap"] = dp_gap
            overall[f"{attr}_eopp_gap"] = eopp_gap

    return overall, rows

overall_metrics, group_metrics = [], []
best_name, best_score, best_pipe = None, -np.inf, None

for name, clf in models.items():
    pipe = Pipeline(steps=[("prep", preprocess), ("clf", clf)])
    pipe.fit(X_train, y_train)

    overall, rows = eval_fairness(name, pipe, X_test, y_test, sens_test)
    overall_metrics.append(overall)
    group_metrics.extend(rows)

    key = "roc_auc" if not np.isnan(overall["roc_auc"]) else "f1"
    score = overall[key]
    if score is not None and (not np.isnan(score)) and score > best_score:
        best_name, best_score, best_pipe = name, score, pipe

# Save metrics
overall_df = pd.DataFrame(overall_metrics)
group_df = pd.DataFrame(group_metrics) if group_metrics else pd.DataFrame(columns=[
    "model","attribute","group","n","p_pos","accuracy","tpr"
])

overall_path = results_dir / "adult_overall_metrics.csv"
group_path = results_dir / "adult_group_metrics.csv"
overall_df.to_csv(overall_path, index=False)
group_df.to_csv(group_path, index=False)

print("Overall metrics:\n", overall_df.round(4))
print("\nSaved:", overall_path)
if not group_df.empty:
    print("\nSample group metrics:\n", group_df.head(12))
    print("Saved:", group_path)

# Save best model
if best_pipe is not None:
    out_model = models_dir / "best_pipeline.joblib"
    joblib.dump(best_pipe, out_model)
    print(f"\nSaved best model: {best_name}  | score={best_score:.4f}  -> {out_model}")


Overall metrics:
     model  accuracy      f1  roc_auc  sex_dp_gap  sex_eopp_gap  race_dp_gap  \
0  logreg    0.8453  0.6521   0.9056      0.1790        0.1369       0.1932   
1      rf    0.8409  0.6575   0.8896      0.1845        0.1059       0.1515   

   race_eopp_gap  
0         0.4449  
1         0.2278  

Saved: C:\Users\hana1\Documents\iva-bias-project\results\metrics\adult_overall_metrics.csv

Sample group metrics:
      model attribute               group     n     p_pos  accuracy       tpr
0   logreg       sex              Female  3681  0.076066  0.916599  0.470199
1   logreg       sex                Male  7625  0.255082  0.810885  0.607067
2   logreg      race  Amer-Indian-Eskimo   106  0.084906  0.924528  0.538462
3   logreg      race  Asian-Pac-Islander   320  0.262500  0.853125  0.717647
4   logreg      race               Black  1044  0.091954  0.899425  0.467153
5   logreg      race               Other   101  0.069307  0.881188  0.272727
6   logreg      race            